In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Parse and Chunk sectional contents from CSA and SIP documents

In [ ]:
pip install pymupdf pandas langchain openai chromadb sentence-transformers langchain_community langchain_chroma --upgrade

In [ ]:
# @title
import os
import re
import fitz  # PyMuPDF
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configuration
PDF_DIRECTORY = "/content/drive/MyDrive/CalWorks/Vector Database/county_reports"  # Set this to your PDF folder path

county_names = [
    "Alameda", "Alpine", "Amador", "Butte", "Calaveras", "Colusa", "Contra Costa",
    "Del Norte", "El Dorado", "Fresno", "Glenn", "Humboldt", "Imperial", "Inyo",
    "Kern", "Kings", "Lake", "Lassen", "Los Angeles", "Madera", "Marin", "Mariposa",
    "Mendocino", "Merced", "Modoc", "Mono", "Monterey", "Napa", "Nevada", "Orange",
    "Placer", "Plumas", "Riverside", "Sacramento", "San Benito", "San Bernardino",
    "San Diego", "San Francisco", "San Joaquin", "San Luis Obispo", "San Mateo",
    "Santa Barbara", "Santa Clara", "Santa Cruz", "Shasta", "Sierra", "Siskiyou",
    "Solano", "Sonoma", "Stanislaus", "Sutter", "Tehama", "Trinity", "Tulare",
    "Tuolumne", "Ventura", "Yolo", "Yuba"
]

# Aliases for counties based on common filename patterns
alias_map = {
    "icdss": "Imperial",
    "lacdss": "Los Angeles",
    "ocss": "Orange",
    "scc": "Santa Clara",
    "sbcs": "San Bernardino",
    # Add more aliases as needed
}

def infer_metadata_from_filename(filename):
    # Clean and normalize
    name_clean = filename.lower().replace("_", " ").replace("-", " ").replace(".pdf", "")
    name_compressed = re.sub(r"\s+", "", name_clean)

    # Report type logic
    if "sip_pr" in name_clean:
        report_type = "Cal-SIP-PR"
    if "sip" in name_clean or "system improvement" in name_clean:
        report_type = "Cal-SIP"
    elif (
        "csa" in name_clean
        or "self-assessment" in name_clean
        or "calworks self-assessment" in name_clean
        or "county self-assessment" in name_clean
        or "fatal flaw" in name_clean
    ):
        report_type = "Cal-CSA"
    else:
        report_type = "Unknown"

    # Try full county match
    normalized_counties = [c.lower().replace(" ", "") for c in county_names]
    county = "Unknown"
    for orig, compressed in zip(county_names, normalized_counties):
        if compressed in name_compressed:
            county = orig
            break

    # If not found, try alias map
    if county == "Unknown":
        for alias, full_name in alias_map.items():
            if alias in name_compressed:
                county = full_name
                break

    return {
        "file": filename,
        "county": county,
        "report_type": report_type
    }


# A Table of Contents is needed
def extract_sections_via_toc(pdf_path, county, report_type, toc_max_pages=5):
    import fitz
    import re

    doc = fitz.open(pdf_path)
    max_pages = len(doc)
    section_entries = []

    toc_lines = []
    for i in range(min(toc_max_pages, len(doc))):
        text = doc[i].get_text()
        lines = text.split("\n")
        for line in lines:
            line = line.strip()
            # Match: Section headers OR named sections like "Introduction", "Executive Summary"
            match = re.match(
                r"^((Section\s+\d+(\.\d+)?[.:]?\s+.+?)|(?:Introduction|Executive Summary|Demographics))\s+\.{3,}\s+(\d{1,3})$",
                line,
                re.IGNORECASE
            )
            if match:
                title = match.group(1).strip()
                page = int(match.group(4))
                toc_lines.append((title, page))

    # Construct section page ranges
    for i, (title, start_page) in enumerate(toc_lines):
        end_page = toc_lines[i + 1][1] - 1 if i + 1 < len(toc_lines) else max_pages
        section_entries.append({
            "county": county,
            "report_type": report_type,
            "section_header": title,
            "start_page": start_page,
            "end_page": end_page
        })

    # Extract section text
    for section in section_entries:
        start = max(0, section["start_page"] - 1)
        end = min(section["end_page"], max_pages)
        text = "".join(doc[p].get_text() for p in range(start, end))
        section["text"] = text.strip()

    return section_entries


def chunk_sections(sections, chunk_size=1000, overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    chunked = []
    for sec in sections:
        if not sec["text"].strip():
            continue
        splits = splitter.split_text(sec["text"])
        for i, chunk in enumerate(splits):
            chunked.append({
                "county": sec["county"],
                "report_type": sec["report_type"],
                "section": sec["section_header"],
                "page": sec["start_page"],
                "chunk_id": f"{sec['section_header']}_chunk{i}",
                "text": chunk
            })
    return chunked

def main():
    pdf_files = [f for f in os.listdir(PDF_DIRECTORY) if f.endswith(".pdf")]
    all_sections = []

    for file in pdf_files:
        meta = infer_metadata_from_filename(file)
        if meta["county"] == "Unknown" or meta["report_type"] == "Unknown":
            print(f"Skipping: {file} (missing county or type)")
            continue
        try:
            path = os.path.join(PDF_DIRECTORY, file)
            sections = extract_sections_via_toc(path, meta["county"], meta["report_type"])

            if len(sections) == 0:
                print(f"No TOC sections found for: {file}")
            else:
                print(f"Found {len(sections)} sections in {file}")
                all_sections.extend(sections)

        except Exception as e:
            print(f"Error processing {file}: {e}")

    chunks = chunk_sections(all_sections)
    print(f"\n{len(all_sections)} sections to {len(chunks)} chunks")

    df = pd.DataFrame(chunks)
    df.to_excel("/content/drive/MyDrive/CalWorks/Vector Database/Output/chunked_sip_csa_output.xlsx", index=False)
    print("Saved to chunked_sip_csa_output.csv")

if __name__ == "__main__":
    main()


# ChromaDb

# Other Free and Efficient Embedding Models

In [4]:
import os
import re
import shutil
import pandas as pd
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings  # Use HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma # Use langchain_community.vectorstores

# Configuration
XLSX_PATH   = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chunked_sip_csa_output.xlsx"
PERSIST_DIR = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]"
COLLECTION  = "sip_csa_chunks"

# Normalize Text
def normalize_text(text: str) -> str:
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"‐|–|—", "-", text)
    text = re.sub(r"“|”|\"|''", '"', text)
    text = re.sub(r"’|‘|`", "'", text)
    return text

# Load Excel Data
df = pd.read_excel(XLSX_PATH).dropna(subset=["text"])
df["chunk_id"] = df.apply(
    lambda row: f"{row['county'].replace(' ', '')}_{row['report_type'].replace('-', '')}_{row['section'].replace(':', '').replace('.', '').replace(' ', '')}_chunk{row.name}",
    axis=1
)
df["text"] = df["text"].apply(normalize_text)
df["section"] = df["section"].astype(str).apply(normalize_text)


# Use HuggingFace embeddings
embedding_func = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Prepare Documents for LangChain's Chroma.from_documents
documents = [
    Document(
        page_content=row["text"],
        metadata={
            "county": row["county"],
            "report_type": row["report_type"],
            "section": row["section"],
            "page": row["page"],
            "chunk_id": row["chunk_id"] # Include chunk_id in metadata
        }
    )
    for _, row in df.iterrows()
]

# Clear the persistence directory if it exists to avoid conflicts
if os.path.exists(PERSIST_DIR):
    print(f"Clearing existing Chroma directory: {PERSIST_DIR}")
    shutil.rmtree(PERSIST_DIR)


print(f"Creating Chroma collection '{COLLECTION}' at '{PERSIST_DIR}'...")

# Create and populate the Chroma collection
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_func,
    collection_name=COLLECTION,
    persist_directory=PERSIST_DIR
)
vectorstore.persist()

print(f"✅ Chroma collection '{COLLECTION}' created and populated.")

# Optional: Verify document count
print(f"Total documents added to collection: {vectorstore._collection.count()}")

/tmp/ipython-input-3688834060.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_func = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating Chroma collection 'sip_csa_chunks' at '/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]'...
✅ Chroma collection 'sip_csa_chunks' created and populated.
Total documents added to collection: 17493


/tmp/ipython-input-3688834060.py:68: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# Optional: Run test query
try:
    result = collection.query(query_texts=["childcare support"], n_results=2)
    print("\nSample Query Result:")
    for i, r in enumerate(result['documents'][0]):
        print(f"Result {i+1}: {r[:300]}...\n---")
except Exception as e:
    print(f"Error during sample query: {e}")


Sample Query Result:
Result 1: County: San Mateo
Section: Section 4. Initial Engagement Analysis

authorization to client (including vouchers, payment, tracking, etc.). Full-time childcare authorization is provided through the Immediate and Continuous Eligibility process. Once a family is referred to WTW, if the family has a chil...
---
Result 2: County: Trinity
Section: Section 4. Initial Engagement Analysis

time, and Trust-line care. Families engaged in any approved activity and where childcare needs are identified have access to Stage One childcare through the aforementioned service provider. Authorization is sent via a referral to HRN. ...
---


In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Setup
PERSIST_DIR = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]"
COLLECTION  = "sip_csa_chunks"

embedding_func = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load vectorstore
vectorstore = Chroma(
    collection_name=COLLECTION,
    persist_directory=PERSIST_DIR,
    embedding_function=embedding_func
)

# Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Run a query
docs = retriever.get_relevant_documents("childcare support")
for i, doc in enumerate(docs):
    print(f"\n--- Document {i+1} ---")
    print(f"Metadata: {doc.metadata}")
    print(f"Content: {doc.page_content[:300]}...")




/tmp/ipython-input-2555424876.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_func = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/tmp/ipython-input-2555424876.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
/tmp/ipython-input-2555424876.py:22: LangChainDeprecationWarning: The method `BaseRetrie


--- Document 1 ---
Metadata: {'section': 'Section 4. Initial Engagement Analysis', 'page': 32, 'county': 'San Mateo', 'report_type': 'Cal-CSA', 'chunk_id': 'SanMateo_CalCSA_Section4InitialEngagementAnalysis_chunk4672'}
Content: authorization to client (including vouchers, payment, tracking, etc.). Full-time childcare authorization is provided through the Immediate and Continuous Eligibility process. Once a family is referred to WTW, if the family has a child who is under the age of 13, or a child of any age who has a disab...

--- Document 2 ---
Metadata: {'report_type': 'Cal-CSA', 'chunk_id': 'SantaClara_CalCSA_Section4InitialEngagementAnalysis_chunk4301', 'county': 'Santa Clara', 'page': 72, 'section': 'Section 4. Initial Engagement Analysis'}
Content: in the forms prior to attending their orientation. Upon receipt, CWES Engagement Unit staff approve child care for a 12 month period from the start date listed on the child care packet. Child care can be provided as early as the start

In [1]:
import chromadb
from chromadb.config import Settings
PERSIST_DIR = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]"

client = chromadb.PersistentClient(
    path=PERSIST_DIR,
    settings=Settings(anonymized_telemetry=False)
)


In [2]:
collections = client.list_collections()

print("📦 Collections and document counts:")
for col in collections:
    # Get the collection object
    c = client.get_collection(col.name)
    print(f"- {col.name}: {c.count()} documents")


📦 Collections and document counts:
- sip_csa_chunks: 17493 documents
